In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

In [3]:
#MNIST 데이터 적재
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL']='2' # 경고 메시지 화면출력 금지

In [6]:
# 파라미터
learning_rate = 0.001 # 신경망 학습률
training_epochs = 100 # 학습 횟수(epoch)
batch_size = 100 # 미니배치의 크기
display_step = 10 # 중간결과 출력 간격

In [7]:
# 신경망 구조 관련 파라미터
n_hidden_1 = 15 # 은닉층의 노드 개수
n_input = 784 # 입력층의 노드 개수 MNIST 데이터(28X28)
n_classes = 10 # 출력층의 노드 수 MNIST 부류 개수(숫자 0~9)

# 텐서 그래프 입력 변수
x = tf.placeholder("float", [None, n_input]) # 입력 : 필기체 영상
y = tf.placeholder("float", [None, n_classes]) # 출력 : 숫자

In [8]:
# 학습모델 MLP정의
def multilayer_perceptron(x, weights, biases):
    # ReLU를 사용하는 은닉층
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # 출력층 (활성화 함수 미사용)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [9]:
# 학습할 파라미터: 가중치(weights), 편차항(biases)
weights = {
    'h1':tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out':tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [11]:
# 신경망 모델 구성, 출력값 pred : 입력 x에 대한 신경망의 출력
pred = multilayer_perceptron(x, weights, biases)

# 비용(오차) 정의 (신경망 출력 pred, 목표 출력 y): 교차엔트로피 사용
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

# 학습 알고리즘 설정
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer() # 변수 초기화 지정

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [14]:
# 데이터 플로우 그래프 실행
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(mnist.train.num_examples / batch_size) # 배치 갯수
    
    for epoch in range(training_epochs): #정해진 횟수 만큼 학습
        avg_cost = 0.
        for i in range(total_batch): # 미니배치
            batch_x, batch_y = mnist.train.next_batch(batch_size) # 적재 
            # 역전파 알고리즘 적용
            _, c = sess.run([optimizer, cost], feed_dict={x:batch_x, y:batch_y})
            avg_cost += c / total_batch # 평군 손실(오류) 계산
            
            if epoch % display_step == 0: # 현재 학습 상황 출력
                print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
        
        # 모델 테스트: out의 최대값 노드와 y 노드가 같으면 정답
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        # correct_prediction 평균
        accurach = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accurach.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 0.061496977
Epoch: 0001 cost= 0.113894650
Epoch: 0001 cost= 0.175400914
Epoch: 0001 cost= 0.236821126
Epoch: 0001 cost= 0.300917986
Epoch: 0001 cost= 0.360016441
Epoch: 0001 cost= 0.413323531
Epoch: 0001 cost= 0.467574699
Epoch: 0001 cost= 0.527087971
Epoch: 0001 cost= 0.581221563
Epoch: 0001 cost= 0.638349311
Epoch: 0001 cost= 0.697592801
Epoch: 0001 cost= 0.751385921
Epoch: 0001 cost= 0.803215408
Epoch: 0001 cost= 0.855146519
Epoch: 0001 cost= 0.903733007
Epoch: 0001 cost= 0.954242925
Epoch: 0001 cost= 1.007094730
Epoch: 0001 cost= 1.064762604
Epoch: 0001 cost= 1.121160018
Epoch: 0001 cost= 1.171724195
Epoch: 0001 cost= 1.224562302
Epoch: 0001 cost= 1.274668752
Epoch: 0001 cost= 1.321937610
Epoch: 0001 cost= 1.365555007
Epoch: 0001 cost= 1.416658675
Epoch: 0001 cost= 1.460132491
Epoch: 0001 cost= 1.502907815
Epoch: 0001 cost= 1.545046092
Epoch: 0001 cost= 1.589962072
Epoch: 0001 cost= 1.634578358
Epoch: 0001 cost= 1.678319106
Epoch: 0001 cost= 1.720640328
Epoch: 000

Epoch: 0001 cost= 6.405042523
Epoch: 0001 cost= 6.411247019
Epoch: 0001 cost= 6.416855392
Epoch: 0001 cost= 6.422696176
Epoch: 0001 cost= 6.429528668
Epoch: 0001 cost= 6.436502452
Epoch: 0001 cost= 6.443037558
Epoch: 0001 cost= 6.449708306
Epoch: 0001 cost= 6.456340384
Epoch: 0001 cost= 6.462333094
Epoch: 0001 cost= 6.467757101
Epoch: 0001 cost= 6.473829911
Epoch: 0001 cost= 6.479989461
Epoch: 0001 cost= 6.485133950
Epoch: 0001 cost= 6.491392897
Epoch: 0001 cost= 6.498004554
Epoch: 0001 cost= 6.504720684
Epoch: 0001 cost= 6.510988760
Epoch: 0001 cost= 6.516794485
Epoch: 0001 cost= 6.522890743
Epoch: 0001 cost= 6.529020464
Epoch: 0001 cost= 6.534344860
Epoch: 0001 cost= 6.539804589
Epoch: 0001 cost= 6.545759183
Epoch: 0001 cost= 6.551577329
Epoch: 0001 cost= 6.557643481
Epoch: 0001 cost= 6.563524781
Epoch: 0001 cost= 6.569848792
Epoch: 0001 cost= 6.575196476
Epoch: 0001 cost= 6.580889671
Epoch: 0001 cost= 6.586014291
Epoch: 0001 cost= 6.591168160
Epoch: 0001 cost= 6.597058773
Epoch: 000

Epoch: 0011 cost= 0.297244772
Epoch: 0011 cost= 0.298987340
Epoch: 0011 cost= 0.300866486
Epoch: 0011 cost= 0.302480453
Epoch: 0011 cost= 0.304436301
Epoch: 0011 cost= 0.306240589
Epoch: 0011 cost= 0.308349086
Epoch: 0011 cost= 0.310002000
Epoch: 0011 cost= 0.311711344
Epoch: 0011 cost= 0.313706613
Epoch: 0011 cost= 0.315759758
Epoch: 0011 cost= 0.317304232
Epoch: 0011 cost= 0.319202548
Epoch: 0011 cost= 0.321230519
Epoch: 0011 cost= 0.323304808
Epoch: 0011 cost= 0.325190680
Epoch: 0011 cost= 0.326885133
Epoch: 0011 cost= 0.328515728
Epoch: 0011 cost= 0.330196302
Epoch: 0011 cost= 0.331891371
Epoch: 0011 cost= 0.333527324
Epoch: 0011 cost= 0.335530218
Epoch: 0011 cost= 0.337602869
Epoch: 0011 cost= 0.339575564
Epoch: 0011 cost= 0.341204834
Epoch: 0011 cost= 0.342795842
Epoch: 0011 cost= 0.344632154
Epoch: 0011 cost= 0.346700947
Epoch: 0011 cost= 0.348268724
Epoch: 0011 cost= 0.350439520
Epoch: 0011 cost= 0.352147193
Epoch: 0011 cost= 0.354422842
Epoch: 0011 cost= 0.356134269
Epoch: 001

Epoch: 0011 cost= 0.962469463
0.6705
0.7407
0.786
0.8135
0.832
0.8466
0.8548
0.8624
0.8707
0.8751
Epoch: 0021 cost= 0.000887110
Epoch: 0021 cost= 0.002121662
Epoch: 0021 cost= 0.002793775
Epoch: 0021 cost= 0.003651499
Epoch: 0021 cost= 0.004420358
Epoch: 0021 cost= 0.005147508
Epoch: 0021 cost= 0.005818969
Epoch: 0021 cost= 0.006448417
Epoch: 0021 cost= 0.007475797
Epoch: 0021 cost= 0.008234826
Epoch: 0021 cost= 0.009299091
Epoch: 0021 cost= 0.009986496
Epoch: 0021 cost= 0.010449204
Epoch: 0021 cost= 0.011247325
Epoch: 0021 cost= 0.012344836
Epoch: 0021 cost= 0.013005641
Epoch: 0021 cost= 0.013747897
Epoch: 0021 cost= 0.014535890
Epoch: 0021 cost= 0.015100265
Epoch: 0021 cost= 0.015835493
Epoch: 0021 cost= 0.016503877
Epoch: 0021 cost= 0.017135555
Epoch: 0021 cost= 0.017927377
Epoch: 0021 cost= 0.018955680
Epoch: 0021 cost= 0.019887292
Epoch: 0021 cost= 0.020651420
Epoch: 0021 cost= 0.021513131
Epoch: 0021 cost= 0.022045014
Epoch: 0021 cost= 0.022521086
Epoch: 0021 cost= 0.023268284
Ep

Epoch: 0021 cost= 0.299971833
Epoch: 0021 cost= 0.300804273
Epoch: 0021 cost= 0.301471287
Epoch: 0021 cost= 0.302340506
Epoch: 0021 cost= 0.302860135
Epoch: 0021 cost= 0.303380631
Epoch: 0021 cost= 0.303924791
Epoch: 0021 cost= 0.304601940
Epoch: 0021 cost= 0.305256129
Epoch: 0021 cost= 0.306475416
Epoch: 0021 cost= 0.307090549
Epoch: 0021 cost= 0.307666458
Epoch: 0021 cost= 0.308156224
Epoch: 0021 cost= 0.309430417
Epoch: 0021 cost= 0.310228282
Epoch: 0021 cost= 0.310889217
Epoch: 0021 cost= 0.311699284
Epoch: 0021 cost= 0.312368237
Epoch: 0021 cost= 0.313043099
Epoch: 0021 cost= 0.313845137
Epoch: 0021 cost= 0.314690771
Epoch: 0021 cost= 0.315589438
Epoch: 0021 cost= 0.316381137
Epoch: 0021 cost= 0.316880792
Epoch: 0021 cost= 0.317683972
Epoch: 0021 cost= 0.318573262
Epoch: 0021 cost= 0.319229660
Epoch: 0021 cost= 0.319917977
Epoch: 0021 cost= 0.321251483
Epoch: 0021 cost= 0.321841272
Epoch: 0021 cost= 0.322718525
Epoch: 0021 cost= 0.323364418
Epoch: 0021 cost= 0.323991476
Epoch: 002

...

Epoch: 0071 cost= 0.078134494
Epoch: 0071 cost= 0.078657345
Epoch: 0071 cost= 0.078900853
Epoch: 0071 cost= 0.079319159
Epoch: 0071 cost= 0.079558743
Epoch: 0071 cost= 0.080093821
Epoch: 0071 cost= 0.080694395
Epoch: 0071 cost= 0.081009526
Epoch: 0071 cost= 0.081310812
Epoch: 0071 cost= 0.081679580
Epoch: 0071 cost= 0.082185439
Epoch: 0071 cost= 0.082536995
Epoch: 0071 cost= 0.082836107
Epoch: 0071 cost= 0.083049993
Epoch: 0071 cost= 0.083609986
Epoch: 0071 cost= 0.083871412
Epoch: 0071 cost= 0.084207614
Epoch: 0071 cost= 0.084610463
Epoch: 0071 cost= 0.084881611
Epoch: 0071 cost= 0.085318958
Epoch: 0071 cost= 0.085504229
Epoch: 0071 cost= 0.085945908
Epoch: 0071 cost= 0.086268137
Epoch: 0071 cost= 0.086702530
Epoch: 0071 cost= 0.087146611
Epoch: 0071 cost= 0.087303242
Epoch: 0071 cost= 0.087903345
Epoch: 0071 cost= 0.088140691
Epoch: 0071 cost= 0.088452838
Epoch: 0071 cost= 0.088737057
Epoch: 0071 cost= 0.089149148
Epoch: 0071 cost= 0.089369043
Epoch: 0071 cost= 0.089538050
Epoch: 007

0.9367
0.9377
0.9373
0.9371
0.9363
0.9368
0.9381
0.939
0.9392
Epoch: 0081 cost= 0.000442287
Epoch: 0081 cost= 0.000783386
Epoch: 0081 cost= 0.001390834
Epoch: 0081 cost= 0.001866061
Epoch: 0081 cost= 0.002449944
Epoch: 0081 cost= 0.002667046
Epoch: 0081 cost= 0.002894249
Epoch: 0081 cost= 0.003028557
Epoch: 0081 cost= 0.003494980
Epoch: 0081 cost= 0.003834529
Epoch: 0081 cost= 0.004020071
Epoch: 0081 cost= 0.004389175
Epoch: 0081 cost= 0.004660583
Epoch: 0081 cost= 0.004865685
Epoch: 0081 cost= 0.004991482
Epoch: 0081 cost= 0.005188585
Epoch: 0081 cost= 0.005332063
Epoch: 0081 cost= 0.005597272
Epoch: 0081 cost= 0.006143740
Epoch: 0081 cost= 0.006511390
Epoch: 0081 cost= 0.006759792
Epoch: 0081 cost= 0.007130948
Epoch: 0081 cost= 0.007383340
Epoch: 0081 cost= 0.007702767
Epoch: 0081 cost= 0.008211807
Epoch: 0081 cost= 0.008353931
Epoch: 0081 cost= 0.008664222
Epoch: 0081 cost= 0.009199123
Epoch: 0081 cost= 0.009557858
Epoch: 0081 cost= 0.009695183
Epoch: 0081 cost= 0.009977500
Epoch: 0

Epoch: 0081 cost= 0.102413600
Epoch: 0081 cost= 0.102619694
Epoch: 0081 cost= 0.102901114
Epoch: 0081 cost= 0.103204953
Epoch: 0081 cost= 0.103316196
Epoch: 0081 cost= 0.103763331
Epoch: 0081 cost= 0.104044769
Epoch: 0081 cost= 0.104368067
Epoch: 0081 cost= 0.104569779
Epoch: 0081 cost= 0.105141507
Epoch: 0081 cost= 0.105360883
Epoch: 0081 cost= 0.105494777
Epoch: 0081 cost= 0.105826486
Epoch: 0081 cost= 0.106514088
Epoch: 0081 cost= 0.106737722
Epoch: 0081 cost= 0.106957025
Epoch: 0081 cost= 0.107211873
Epoch: 0081 cost= 0.107519761
Epoch: 0081 cost= 0.108132098
Epoch: 0081 cost= 0.108483369
Epoch: 0081 cost= 0.108957412
Epoch: 0081 cost= 0.109146069
Epoch: 0081 cost= 0.109668255
Epoch: 0081 cost= 0.109874780
Epoch: 0081 cost= 0.110057867
Epoch: 0081 cost= 0.110284380
Epoch: 0081 cost= 0.110788952
Epoch: 0081 cost= 0.111100775
Epoch: 0081 cost= 0.111386718
Epoch: 0081 cost= 0.111566313
Epoch: 0081 cost= 0.111834247
Epoch: 0081 cost= 0.112275138
Epoch: 0081 cost= 0.112455117
Epoch: 008

Epoch: 0091 cost= 0.042671108
Epoch: 0091 cost= 0.042965404
Epoch: 0091 cost= 0.043126344
Epoch: 0091 cost= 0.043420024
Epoch: 0091 cost= 0.043747331
Epoch: 0091 cost= 0.043937199
Epoch: 0091 cost= 0.044035651
Epoch: 0091 cost= 0.044463583
Epoch: 0091 cost= 0.044888408
Epoch: 0091 cost= 0.045177172
Epoch: 0091 cost= 0.045676933
Epoch: 0091 cost= 0.045904408
Epoch: 0091 cost= 0.046123986
Epoch: 0091 cost= 0.046340012
Epoch: 0091 cost= 0.046956811
Epoch: 0091 cost= 0.047321656
Epoch: 0091 cost= 0.047593945
Epoch: 0091 cost= 0.047916273
Epoch: 0091 cost= 0.048463230
Epoch: 0091 cost= 0.048801406
Epoch: 0091 cost= 0.049109478
Epoch: 0091 cost= 0.049400061
Epoch: 0091 cost= 0.049751519
Epoch: 0091 cost= 0.050033732
Epoch: 0091 cost= 0.050165183
Epoch: 0091 cost= 0.050233781
Epoch: 0091 cost= 0.050462671
Epoch: 0091 cost= 0.050672640
Epoch: 0091 cost= 0.051040882
Epoch: 0091 cost= 0.051485827
Epoch: 0091 cost= 0.051749078
Epoch: 0091 cost= 0.051836569
Epoch: 0091 cost= 0.052483306
Epoch: 009

Epoch: 0091 cost= 0.166892260
Epoch: 0091 cost= 0.167465891
Epoch: 0091 cost= 0.167645125
Epoch: 0091 cost= 0.167830182
Epoch: 0091 cost= 0.168126253
Epoch: 0091 cost= 0.168363745
Epoch: 0091 cost= 0.168914250
Epoch: 0091 cost= 0.169171360
Epoch: 0091 cost= 0.169436980
Epoch: 0091 cost= 0.169718698
Epoch: 0091 cost= 0.169947005
0.9397
0.9393
0.941
0.9423
0.9418
0.9424
0.9431
0.9431
0.9427
0.9418


In [ ]:
'''
* 프로그램에서 사용된 API function에 대한 설명

위 프로그램은 다층 퍼셉트론을 MNIST 데이터 분류에 적용한 프로그램이다.
위 프로그램에서 구성한 신경망은 은닉층 한 개와 출력층으로 되어 있으며, 
은닉층은 15개의 노드를 갖도록 한다. 입력 데이터의 크기가 28X28(=784)이므로 입력층에는 784개의 노드를 만든다.
출력에서 0부터 9까지 각 숫자에 대한 확률을  출력하도록 10개의 노드를 만든다.

27행의 multilayer_perceptron 함수를 사용하여 다층 퍼셉트론을 구성하는데, 은닉층의 활성화 함수로 ReLU를 사용하도록 설정한다.
출력층의 값을 소프트맥스를 사용하도록 하고, 오차 함수로 교차엔트로피를 사용하도록 지정한다.
최적화 방법으로 Adam을 사용하도록 지정한다.
프로그램을 실행하면, 학습 오차를 나타내는 교차엔트로피 값이 점점 감소하는 것을 확인할 수 있다.


'''